<a href="https://colab.research.google.com/github/Zitze-Wizz/Kaggle-Challenge/blob/master/Final_0_66_Kaggle_Competition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import imblearn

import numpy as np
import pandas as pd

import datetime

from sklearn.model_selection import train_test_split,  cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix, classification_report, f1_score

import matplotlib.pyplot as plt
import seaborn as sns

from imblearn.over_sampling import SMOTENC # for oversampling the minority class, i.e generating synthetic samples
from imblearn.under_sampling import RandomUnderSampler # for undersampling the majority class

In [ ]:
gdrive_data_url = "https://drive.google.com/file/d/1h1WiB50t_oosZgPZ5CTfWzwEVGKf0Jek/view?usp=drive_link"
file_id = gdrive_data_url.split('/')[-2]
data_file='https://drive.google.com/uc?export=download&id=' + file_id
X = pd.read_csv(data_file, na_values=['?', ''])

In [ ]:
gdrive_data_url = "https://drive.google.com/file/d/1h38t8SGpBU2rcaXidnB6dNuSeoN3HI2F/view?usp=sharing"
file_id = gdrive_data_url.split('/')[-2]
data_file='https://drive.google.com/uc?export=download&id=' + file_id
y = pd.read_csv(data_file, na_values=['?', ''])

In [ ]:
gdrive_data_url = "https://drive.google.com/file/d/1KTgc8k4WtBj6HJDfSISTmLZxuVcjNttN/view?usp=sharing"
file_id = gdrive_data_url.split('/')[-2]
data_file='https://drive.google.com/uc?export=download&id=' + file_id
X_final = pd.read_csv(data_file, na_values=['?', ''])

In [ ]:
y.head()

,Id,Expected
0,0,0
1,1,0
2,2,2
3,3,1
4,4,2


In [ ]:
X2 = X[["Id", "98","11","50","24","61","20","72","62","87","60","86","74","42","37","2","6","63","53", "91"]]  # 0.64
# X2 = X[["Id", "98","11","50","24","61","20","72","62","87","60","86","42","37","6"]] #,"34","2","74"]] # 0.64
# X2 = X[["Id", "98","11","50","24","61","20","72","62","87","60"]] # ,"86","42","37","6","34","2","74"]]
#X2 = X[["Id", "98","11","50","24","61","20","72","62","87","60","86","42","37","6","34","2","74"]]
#X2 = X[["Id", "98","11","50","24","61","20","72","62","87","60","86","42","37","6","34","2","74"]]

X = X2 #[:1000]
y = y #[:1000]
X.shape


(10000, 21)

In [ ]:
y = y.drop (columns = ["Id"])

In [ ]:
X.head()

,Id,98,11,50,24,61,20,72,62,87,...,86,42,37,6,34,2,74,42,37,2
0,0,2.038736,2.280097,0.063419,-4.479576,0.467676,-1.015441,-0.472142,16.306059,1.871807,...,-0.315395,-6.725756,-3.434408,29.897623,23.808472,7.020637,5.125120,-6.725756,-3.434408,7.020637
1,1,0.413158,-5.775092,39.349176,0.764284,0.008065,-2.875074,2.106962,4.646883,7.049593,...,-1.230639,-2.306467,4.490605,-28.162402,5.738793,22.176950,0.664921,-2.306467,4.490605,22.176950
2,2,3.701254,-5.828967,0.112884,0.950084,0.339700,4.494041,-7.420275,94.645030,0.136055,...,6.344787,4.533684,-3.971667,-22.016388,19.629010,1.663984,91.770769,4.533684,-3.971667,1.663984
3,3,5.892810,-4.796236,1484.131591,-1.805663,1.581812,2.786869,2.506215,14.622529,10.358130,...,-2.016885,0.208164,-6.897755,12.132919,25.526839,92.827212,7.526305,0.208164,-6.897755,92.827212
4,4,2.157437,-6.056380,0.212263,0.228554,0.399809,-4.401633,-4.651395,8.093200,60.634258,...,3.523166,-3.933762,6.709862,-11.135888,5.240953,4.823582,1.461310,-3.933762,6.709862,4.823582


In [ ]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 21 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Id      10000 non-null  int64  
 1   98      10000 non-null  float64
 2   11      10000 non-null  float64
 3   50      10000 non-null  float64
 4   24      10000 non-null  float64
 5   61      10000 non-null  float64
 6   20      10000 non-null  float64
 7   72      10000 non-null  float64
 8   62      10000 non-null  float64
 9   87      10000 non-null  float64
 10  60      10000 non-null  float64
 11  86      10000 non-null  float64
 12  42      10000 non-null  float64
 13  37      10000 non-null  float64
 14  6       10000 non-null  float64
 15  34      10000 non-null  float64
 16  2       10000 non-null  float64
 17  74      10000 non-null  float64
 18  42      10000 non-null  float64
 19  37      10000 non-null  float64
 20  2       10000 non-null  float64
dtypes: float64(20), int64(1)
memory usag

In [ ]:
X.columns

Index(['Id', '98', '11', '50', '24', '61', '20', '72', '62', '87', '60', '86',
       '42', '37', '6', '34', '2', '74', '42', '37', '2'],
      dtype='object')

In [ ]:
'''nulls = pd.DataFrame(df.isna().sum(), columns = ["value"])
nulls[nulls["value"] != 0]'''

'nulls = pd.DataFrame(df.isna().sum(), columns = ["value"])\nnulls[nulls["value"] != 0]'

In [ ]:
X.describe()

,Id,98,11,50,24,61,20,72,62,87,...,86,42,37,6,34,2,74,42,37,2
count,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,1.000000e+04,10000.000000,10000.000000,1.000000e+04,1.000000e+04,...,10000.000000,10000.000000,10000.000000,10000.000000,1.000000e+04,1.000000e+04,10000.000000,10000.000000,10000.000000,1.000000e+04
mean,4999.50000,0.031468,0.018512,194.853540,0.218314,1.628473e+01,0.044056,-0.019336,1.322092e+01,9.615469e+00,...,-0.027733,-0.010332,-0.052100,0.520059,9.588271e+00,1.323895e+01,144.332411,-0.010332,-0.052100,1.323895e+01
std,2886.89568,3.231366,3.680990,413.031326,3.128920,3.854573e+01,3.528149,3.746007,1.976919e+01,1.460689e+01,...,3.405446,3.344346,3.183651,31.791384,1.366015e+01,1.870323e+01,360.580276,3.344346,3.183651,1.870323e+01
min,0.00000,-16.112369,-14.056768,0.000054,-12.793088,5.325880e-07,-13.921941,-14.739145,4.791200e-07,6.991405e-07,...,-14.481581,-12.981847,-11.866657,-129.141171,8.202144e-09,7.274511e-07,0.000011,-12.981847,-11.866657,7.274511e-07
25%,2499.75000,-2.116268,-2.405063,1.593135,-1.890231,9.212007e-02,-2.256933,-2.523641,1.239233e+00,9.117398e-01,...,-2.352000,-2.301496,-2.199275,-21.037225,9.776764e-01,1.318718e+00,0.658136,-2.301496,-2.199275,1.318718e+00
50%,4999.50000,0.035511,0.023908,12.896343,0.166319,7.561957e-01,0.028249,-0.074143,5.712324e+00,4.075308e+00,...,-0.033018,0.013327,-0.076071,0.351341,4.326855e+00,6.109321e+00,6.049327,0.013327,-0.076071,6.109321e+00
75%,7499.25000,2.156595,2.461454,107.897505,2.306593,6.717083e+00,2.382634,2.457654,1.675450e+01,1.226529e+01,...,2.238050,2.265405,2.105626,21.714889,1.246799e+01,1.758705e+01,57.847220,2.265405,2.105626,1.758705e+01
max,9999.00000,12.174668,13.241661,1484.131591,11.455826,1.484132e+02,14.008867,15.720605,2.766526e+02,1.908295e+02,...,14.183615,12.992755,12.916188,118.399310,1.820583e+02,2.681004e+02,1484.131591,12.992755,12.916188,2.681004e+02


In [ ]:
'''X.hist (figsize = (15, 10)) # histogram for all numeric features'''

'X.hist (figsize = (15, 10)) # histogram for all numeric features'

In [ ]:
X.nunique()

Id    10000
98    10000
11    10000
50     9319
24    10000
61     9404
20    10000
72    10000
62    10000
87    10000
60     9161
86    10000
42    10000
37    10000
6     10000
34    10000
2     10000
74     9515
42    10000
37    10000
2     10000
dtype: int64

In [ ]:
'''high_corr = df.corr()[str(i)][df.corr()[str(i)] > 0.4]
    print(i, high_corr)'''

'high_corr = df.corr()[str(i)][df.corr()[str(i)] > 0.4]\n    print(i, high_corr)'

In [ ]:
'''import seaborn as sns

r = X_train.corr()
r[abs(r) <= 0.5] = 0
plt.figure(figsize=(30, 30))
sns.heatmap(r, annot=True, cmap='coolwarm');'''

"import seaborn as sns\n\nr = X_train.corr()\nr[abs(r) <= 0.5] = 0\nplt.figure(figsize=(30, 30))\nsns.heatmap(r, annot=True, cmap='coolwarm');"

## Split into Train and Test Data

In [ ]:
y.value_counts()

Expected
2           3365
0           3341
1           3294
dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)           #Stratify: Enough y in each set train & test
X_train.shape, X_test.shape

((7000, 21), (3000, 21))

In [ ]:
'''from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder


xgb = XGBClassifier(random_state=42)

pipeline_xgb = Pipeline([("model", xgb)])

le = LabelEncoder()
# transform y labels into 0 and 1 as latest xgboost version doesn't support string labels
y_train_le = le.fit_transform(y_train)
y_test_le = le.fit_transform(y_test)'''

'from xgboost import XGBClassifier\nfrom sklearn.preprocessing import LabelEncoder\n\n\nxgb = XGBClassifier(random_state=42)\n\npipeline_xgb = Pipeline([("model", xgb)])\n\nle = LabelEncoder()\n# transform y labels into 0 and 1 as latest xgboost version doesn\'t support string labels\ny_train_le = le.fit_transform(y_train)\ny_test_le = le.fit_transform(y_test)'

In [ ]:
y_train.shape

(7000, 1)

In [ ]:
# Fit

from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state=42).fit(X_train, y_train)
y_pred = model.predict(X_test)

<ipython-input-96-32dab62745b4>:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model = RandomForestClassifier(random_state=42).fit(X_train, y_train)


In [ ]:
model.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}

In [ ]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.61      0.64      0.63      1002
           1       0.68      0.70      0.69       988
           2       0.64      0.59      0.61      1010

    accuracy                           0.64      3000
   macro avg       0.64      0.64      0.64      3000
weighted avg       0.64      0.64      0.64      3000



In [ ]:
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score

clf = RandomForestClassifier(random_state=42)                   # 0.64

random_search = {
                  'criterion': ['entropy', 'gini'],
                  'max_depth': [10, 20, None],
                  'max_features': ['sqrt', 'log2'],
                  'n_estimators': [50, 100, 200]
                 }



random_search_obj = RandomizedSearchCV(estimator=clf,
                           param_distributions=random_search,
                           n_iter=10, # total number of models it will try out by random selections
                           scoring='f1_macro',
                           cv=3, verbose=1, random_state=42, n_jobs=-1)

random_search_obj.fit(X_train, y_train)

best_model = random_search_obj.best_estimator_

predictionforest = best_model.predict(X_test)
print(classification_report(y_test,predictionforest))

best_model.get_params()

Fitting 3 folds for each of 10 candidates, totalling 30 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:909: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


              precision    recall  f1-score   support

           0       0.63      0.65      0.64      1002
           1       0.69      0.71      0.70       988
           2       0.66      0.62      0.64      1010

    accuracy                           0.66      3000
   macro avg       0.66      0.66      0.66      3000
weighted avg       0.66      0.66      0.66      3000



{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'log2',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 200,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}

In [ ]:
from sklearn.metrics import f1_score

f1 = f1_score(y_test, predictionforest, average='macro')
f1

0.6590238431253711

In [ ]:
best_model.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'log2',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 200,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}

In [ ]:
'''clf2 = RandomForestClassifier(random_state=42)

random_search2 = {
                  'ccp_alpha' : [0],
                  'criterion': ['gini'],                     # 'entropy',
                  'max_depth': [20],                         # 20
                  'max_features': ["sqrt"],                  # 'log2'
                  'n_estimators': [150, 155, 160, 165, 170],           #around 140
                  'max_samples': [None],
                  'min_samples_leaf': [3,7,10],

                 }

               # 2 x 3 x 2 x 3 = 36 => randomly selects 10 combinations (models) from 36 possible model hyperparameter configs
               # 1. RandomForestClassifier(criterion='gini', max_depth=10, max_features='sqrt', n_estimators=50)
               # 2. RandomForestClassifier(criterion='entropy', max_depth=20, max_features='log2', n_estimators=200)
               # ... total of n_iter models




random_search_obj2 = RandomizedSearchCV(estimator=clf,
                           param_distributions=random_search2,
                           n_iter=20, # total number of models it will try out by random selections
                           scoring='f1_macro',
                           cv=3, verbose=1, random_state=42, n_jobs=-1)

random_search_obj2.fit(X_train, y_train)

best_model2 = random_search_obj2.best_estimator_

predictionforest2 = best_model2.predict(X_test)
print(classification_report(y_test,predictionforest2))

best_model2.get_params()


#nestim = 100m maxf = sqrt, mdepth = 20'''

'clf2 = RandomForestClassifier(random_state=42)\n\nrandom_search2 = {\n                  \'ccp_alpha\' : [0], \n                  \'criterion\': [\'gini\'],                     # \'entropy\', \n                  \'max_depth\': [20],                         # 20\n                  \'max_features\': ["sqrt"],                  # \'log2\'\n                  \'n_estimators\': [150, 155, 160, 165, 170],           #around 140\n                  \'max_samples\': [None],\n                  \'min_samples_leaf\': [3,7,10],\n                 \n                 }\n\n               # 2 x 3 x 2 x 3 = 36 => randomly selects 10 combinations (models) from 36 possible model hyperparameter configs\n               # 1. RandomForestClassifier(criterion=\'gini\', max_depth=10, max_features=\'sqrt\', n_estimators=50)\n               # 2. RandomForestClassifier(criterion=\'entropy\', max_depth=20, max_features=\'log2\', n_estimators=200)\n               # ... total of n_iter models\n\n\n\n\nrandom_search_obj2 

In [ ]:
'''clf3 = RandomForestClassifier(random_state=42)

random_search3 = {
                  'criterion': ['entropy', 'gini'],
                  'max_depth': [10, 20, None],
                  'max_features': ['sqrt', 'log2'],
                  'n_estimators': [50, 100, 200]
                 }
               # 2 x 3 x 2 x 3 = 36 => randomly selects 10 combinations (models) from 36 possible model hyperparameter configs
               # 1. RandomForestClassifier(criterion='gini', max_depth=10, max_features='sqrt', n_estimators=50)
               # 2. RandomForestClassifier(criterion='entropy', max_depth=20, max_features='log2', n_estimators=200)
               # ... total of n_iter models




random_search_obj3 = RandomizedSearchCV(estimator=clf,
                           param_distributions=random_search3,
                           n_iter=20, # total number of models it will try out by random selections
                           scoring='f1_macro',
                           cv=3, verbose=1, random_state=42, n_jobs=-1)

random_search_obj3.fit(X_train, y_train)

best_model3 = random_search_obj3.best_estimator_

predictionforest3 = best_model3.predict(X_test)
print(classification_report(y_test,predictionforest3))

best_model3.get_params()'''

"clf3 = RandomForestClassifier(random_state=42)\n\nrandom_search3 = {\n                  'criterion': ['entropy', 'gini'],\n                  'max_depth': [10, 20, None],\n                  'max_features': ['sqrt', 'log2'],\n                  'n_estimators': [50, 100, 200]\n                 }\n               # 2 x 3 x 2 x 3 = 36 => randomly selects 10 combinations (models) from 36 possible model hyperparameter configs\n               # 1. RandomForestClassifier(criterion='gini', max_depth=10, max_features='sqrt', n_estimators=50)\n               # 2. RandomForestClassifier(criterion='entropy', max_depth=20, max_features='log2', n_estimators=200)\n               # ... total of n_iter models\n\n\n\n\nrandom_search_obj3 = RandomizedSearchCV(estimator=clf,\n                           param_distributions=random_search3,\n                           n_iter=20, # total number of models it will try out by random selections\n                           scoring='f1_macro',\n                       

In [ ]:
'''clf4 = RandomForestClassifier(random_state=42)

random_search4 = {
                  'ccp_alpha' : [0],
                  'criterion': ['gini'],    # 'entropy',
                  'max_depth': [23,25,27, None],      # 10
                  'max_features': [4],           # 'log2'
                  'n_estimators': [110, 120, 130],
                  'max_samples': [1500, 2000, 2500]
                 }

               # 2 x 3 x 2 x 3 = 36 => randomly selects 10 combinations (models) from 36 possible model hyperparameter configs
               # 1. RandomForestClassifier(criterion='gini', max_depth=10, max_features='sqrt', n_estimators=50)
               # 2. RandomForestClassifier(criterion='entropy', max_depth=20, max_features='log2', n_estimators=200)
               # ... total of n_iter models




random_search_obj4 = RandomizedSearchCV(estimator=clf,
                           param_distributions=random_search4,
                           n_iter=20, # total number of models it will try out by random selections
                           scoring='f1_macro',
                           cv=3, verbose=1, random_state=42, n_jobs=-1)

random_search_obj4.fit(X_train, y_train)

best_model4 = random_search_obj4.best_estimator_

predictionforest4 = best_model4.predict(X_test)
print(classification_report(y_test,predictionforest4))

best_model4.get_params()'''

"clf4 = RandomForestClassifier(random_state=42)\n\nrandom_search4 = {\n                  'ccp_alpha' : [0], \n                  'criterion': ['gini'],    # 'entropy', \n                  'max_depth': [23,25,27, None],      # 10\n                  'max_features': [4],           # 'log2'\n                  'n_estimators': [110, 120, 130],\n                  'max_samples': [1500, 2000, 2500]\n                 }\n\n               # 2 x 3 x 2 x 3 = 36 => randomly selects 10 combinations (models) from 36 possible model hyperparameter configs\n               # 1. RandomForestClassifier(criterion='gini', max_depth=10, max_features='sqrt', n_estimators=50)\n               # 2. RandomForestClassifier(criterion='entropy', max_depth=20, max_features='log2', n_estimators=200)\n               # ... total of n_iter models\n\n\n\n\nrandom_search_obj4 = RandomizedSearchCV(estimator=clf,\n                           param_distributions=random_search4,\n                           n_iter=20, # total number

In [ ]:
X_final2 = X_final[["Id", "98","11","50","24","61","20","72","62","87","60","86","74","42","37","2","6","63","53", "91"]]  # 0.64
# X2 = X[["Id", "98","11","50","24","61","20","72","62","87","60","86","42","37","6"]] #,"34","2","74"]] # 0.64
# X2 = X[["Id", "98","11","50","24","61","20","72","62","87","60"]] # ,"86","42","37","6","34","2","74"]]
#X2 = X[["Id", "98","11","50","24","61","20","72","62","87","60","86","42","37","6","34","2","74"]]
#X2 = X[["Id", "98","11","50","24","61","20","72","62","87","60","86","42","37","6","34","2","74"]]

X_final = X_final2 #[:1000]
X_final.shape, X_final.head()


((20000, 20),
    Id        98        11           50        24         61         20  \
 0   0  1.627644 -5.475461     0.278009 -0.885763   2.193392   1.396366   
 1   1 -5.143795  1.160366     2.754611  1.870688   0.172501 -10.584147   
 2   2 -1.672424 -1.802028    33.041780 -3.256410   0.547314  -7.137388   
 3   3 -0.704601 -6.004230  1484.131591 -1.801898  23.632765   1.907041   
 4   4 -0.988028  2.263138   108.464776  0.366459   5.223058  -4.322174   
 
          72         62         87         60        86           74        42  \
 0 -0.779777  71.459550   5.907949   0.008329  1.157463     0.118797 -1.329417   
 1  0.767247   0.002066  18.881935  48.102792  0.783257  1484.131591  3.385397   
 2 -2.056834   2.417599   0.976244   0.491529  1.117406    13.434883 -0.222840   
 3 -3.601637  35.067343   0.083456  10.613394 -1.239871     4.351740 -1.925966   
 4  3.782404   0.122652   0.690354   0.112896  4.020908     9.081033 -2.211748   
 
          37          2          6      

In [ ]:
X_final.head()

,Id,98,11,50,24,61,20,72,62,87,60,86,74,42,37,2,6,63,53,91
0,0,1.627644,-5.475461,0.278009,-0.885763,2.193392,1.396366,-0.779777,71.459550,5.907949,0.008329,1.157463,0.118797,-1.329417,5.515789,9.531945,-23.881055,303.998918,0.550952,-0.127226
1,1,-5.143795,1.160366,2.754611,1.870688,0.172501,-10.584147,0.767247,0.002066,18.881935,48.102792,0.783257,1484.131591,3.385397,-1.109945,13.064256,-47.774112,12.655579,0.614540,0.297756
2,2,-1.672424,-1.802028,33.041780,-3.256410,0.547314,-7.137388,-2.056834,2.417599,0.976244,0.491529,1.117406,13.434883,-0.222840,-4.953978,10.370677,65.747722,191.214709,6.790447,0.194697
3,3,-0.704601,-6.004230,1484.131591,-1.801898,23.632765,1.907041,-3.601637,35.067343,0.083456,10.613394,-1.239871,4.351740,-1.925966,1.836020,0.379516,-11.036037,825.842853,45.208456,2.435104
4,4,-0.988028,2.263138,108.464776,0.366459,5.223058,-4.322174,3.782404,0.122652,0.690354,0.112896,4.020908,9.081033,-2.211748,-1.147118,0.535448,8.185199,39.186811,0.469137,-1.842170


In [ ]:
y_final = best_model.predict(X_final)

ValueError: ignored

In [ ]:
X_final.columns

In [ ]:
X_train.columns

In [ ]:
IDs = X_final["Id"]

ydf = pd.DataFrame(IDs)
ydf["Predicted"] = y_final

ydf.head()
#ydf.shape

In [ ]:
from google.colab import files

# Save the DataFrame as a CSV file
ydf.to_csv('ydf.csv', index=False)

# Download the CSV file


#files.download('ydf.csv')

In [ ]:
f1